In [ ]:
import os
import sys

import pandas as pd
import datetime as dt
import plotly
import plotly.express as px
import plotly.graph_objects as go
import logging
import json
import gc
import requests

# used for getting forex exchange rates and not critical to verify SSL
requests.packages.urllib3.disable_warnings() 

cur_dir = os.getcwd()
SRC_PATH = cur_dir[: cur_dir.index("fortunato-wheels-engine") + len("fortunato-wheels-engine")]
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

from src.websites.kijiji import connect_to_database

%load_ext autoreload
%autoreload 2

# Connecting to DB and loading data

In [ ]:
client, db, collection = connect_to_database()

In [ ]:
collection.count_documents({})

In [ ]:
all_ads_raw = pd.DataFrame(collection.find())

In [ ]:
all_ads_raw.info()

In [ ]:
all_ads_raw.describe()

In [ ]:
def preprocess_raw_ad_data(raw_ad_data):

    preprocessed_ads = raw_ad_data.assign(
        created_date_time = pd.to_datetime(raw_ad_data.created, unit="s", origin="unix"),
        modified_date_time = pd.to_datetime(raw_ad_data.modified, unit="s", origin="unix"),
        year = raw_ad_data.year.astype("Int64"),
    )

    preprocessed_ads['age_at_posting'] = preprocessed_ads.created_date_time.dt.year - preprocessed_ads.year

    preprocessed_ads["mileage_per_year"] = preprocessed_ads.mileage / preprocessed_ads.age_at_posting
    
    return preprocessed_ads

In [ ]:
ads = preprocess_raw_ad_data(all_ads_raw)

In [ ]:
ads.info()

In [ ]:
fig = px.histogram(
    ads,
    x="created_date_time",
    # y="price",
)
fig.show()

In [ ]:
ads.url.tail()

# Focussing on Honda CRV and Toyota Rav4

In [ ]:
ads_focus = ads.query("((make == 'Toyota') & (model == 'RAV 4')) | ((make == 'Honda') & (model == 'CR-V'))")

Analytical question: What is the difference in price between the two cars?

In [ ]:
fig = px.violin(
    ads_focus.query("make == 'Toyota'"), 
    y="price", 
    x="model",
    color="age_at_posting",  
    box=True, 
    points="outliers", 
    # hover_data=ads_focus.columns
    category_orders={"age_at_posting": sorted(ads_focus['age_at_posting'].unique())}
)
fig.show()

In [ ]:
toyotas = ads_focus.query("make == 'Toyota'")
fig = px.scatter(
    y=toyotas.price, 
    x=toyotas.age_at_posting,
    trendline="lowess",
    color=toyotas.date_time.dt.year.astype("float"),  
    color_continuous_scale="viridis"
).update_traces(marker=dict(
            # color='darkblue',
            opacity=0.7,
            size=10,
            # line=dict(
            #     color='MediumPurple',
            #     width=2
            # )
            )
)
fig.show()

# Inspecting Kaggle Data

Issue is it doesn't have posting date so don't know age of car when posted with price, not very useful.

In [ ]:
raw_kaggle = pd.read_csv(os.path.join(os.pardir, "data", "raw", "kaggle-vehicles.csv"), 
    parse_dates = ["posting_date"]
)

In [ ]:
raw_kaggle.info()

In [ ]:
raw_kaggle.posting_date.tail()

In [ ]:
def preprocess_kaggle_data(raw_kaggle_data):

    preprocessed_kaggle = raw_kaggle_data.assign(
        posting_date = pd.to_datetime(raw_kaggle_data.posting_date, utc=True),
    )
    
    return preprocessed_kaggle

In [ ]:
kaggle = preprocess_kaggle_data(raw_kaggle)

In [ ]:
# plot price vs posting date
fig = px.scatter(
    kaggle.loc[kaggle.price < 1_000_000],
    x="posting_date",
    y="price",
    color="manufacturer",
)
fig.show()

# Inspecting Car Guru Data

## EDA

1.  **Formulate your question** - sharp and pointed, figure out if right data to answer
    - What is the distribution of prices for Honda CRV and Toyota Rav4 given how old it is when posted for sale?
    - What is the distribution of mileage per year for all Rav4/CRV's?
    - How old of a used Rav4/CRV can I buy for $20k?
    - What features of a Rav4/CRV make it more expensive?
2.  **Read in your data**
3.  **Check the packaging** - # rows/cols, nulls, dtypes
4.  **Look at the top and the bottom of your data**
5.  **Check your “n”s** - expected values actually there? Recorded as expected?
6.  **Validate with at least one external data source** - check rough alignment against any values
7.  **Make a plot** - creates expectations and shows deviation from those
8.  **Try the easy solution first & Challenge the Solution** - simplest and if no trends shown challenge assumptions
9.  **Follow up Questions** - The right data? Need other data? The right question w/ impact?


In [ ]:
raw_carguru = pd.read_csv(os.path.join(os.pardir, "data", "raw", "cargurus-vehicle-ads.csv", ), 
    parse_dates = ["listed_date"],
    nrows=50_000
)

In [ ]:
print(f"Number of ads: {raw_carguru.shape[0]}")
print(f"Number of columns: {raw_carguru.shape[1]}")

In [ ]:
raw_carguru.info(memory_usage = "deep")

In [ ]:
raw_carguru.head()

In [ ]:
raw_carguru.tail()

In [ ]:
fig = px.imshow(
    raw_carguru.isnull().T,
    title="Missing Values in Carguru's Dataset <br><sup>Red indicates a missing value, zoom in as far as you want to see</sup>",
    labels=dict(
        variable="Column",
        value="Missing values",
        color="Missing values"
    ),
    height = 1400,
    color_continuous_scale='OrRd',
    binary_compression_level = 9
).update_layout(
    showlegend=False,
    coloraxis_showscale=False
)
fig.show()

In [ ]:
# calculate percentage of each column that is null
print((raw_carguru.isnull().sum() / raw_carguru.shape[0]).sort_values(ascending=False).head(20)*100)

In [ ]:
# plot how many ads there are by the top 30 make_name values
fig = px.histogram(
    raw_carguru.loc[raw_carguru.make_name.isin(raw_carguru.make_name.value_counts().index[:15])],
    x="make_name",
    title="Number of ads by Make (Top 15)",
    color="make_name",
    labels={"make_name": "Make"    },
    color_discrete_sequence=px.colors.qualitative.Dark24,
    height=500,
    category_orders={"make_name": raw_carguru.make_name.value_counts().index[:15]}
)
fig.show()

In [ ]:
#plot scatter of price and listing_date
fig = px.scatter(
    raw_carguru,
    title="Price vs Listing Date",
    x="listed_date",
    y="price",
    labels = {
        "price": "Price ($)",
        "listed_date": "Date Listed"
    },
    render_mode='webgl'
)
fig.show()

In [ ]:
# plot distribution of price
fig = px.histogram(
    raw_carguru,
    x="price",
    title="Distribution of Price",
    labels={
        "price": "Price ($)",
        "count": "Number of Ads"
    }
)
fig.show()

In [ ]:
# how many car ads are more than 200k?
print(f"Number of cars posted for >$200k: {len(raw_carguru.query('price > 200_000'))} out of {len(raw_carguru)}")

In [ ]:
# plot distribution of cars with price < 200k
fig = px.histogram(
    raw_carguru.query("price < 200_000"),
    x="price",
    title="Distribution of Vehicle Prices < $200k",
    labels={
        "price": "Price ($)",
        "count": "Number of Ads"
    }
)
fig.add_vline(
    x=raw_carguru.query("price < 200_000").price.median(), 
    line_dash = 'dash', 
    line_color = 'firebrick',
    annotation_text = f" Median: ${raw_carguru.query('price < 200_000').price.median():.0f}",
)
fig.update_traces(
    xbins_size = 500
)
fig.show()

In [ ]:
raw_carguru["age_at_posting"] = raw_carguru.listed_date.dt.year - raw_carguru.year
raw_carguru["mileage_per_year"] = raw_carguru.mileage / raw_carguru.age_at_posting

In [ ]:
# plot distribution of mileage per year
fig = px.histogram(
    raw_carguru.query("(mileage_per_year > 0) & (mileage_per_year < 100_000)"),
    x="mileage_per_year",
    title="Distribution of Mileage per Year",
    labels={
        "mileage_per_year": "Mileage per Year (mi)",
        "count": "Number of Ads"
    }
)
fig.add_vline(
    x=raw_carguru.query("(mileage_per_year > 0) & (mileage_per_year < 100_000)").mileage_per_year.median(), 
    line_dash = 'dash', 
    line_color = 'firebrick',
    annotation_text = f" Median: {raw_carguru.query('(mileage_per_year > 0) & (mileage_per_year < 100_000)').mileage_per_year.median():.0f} mi/year",
)
fig.show()

In [ ]:
# plot effect of age at posting on price
fig = px.scatter(
    raw_carguru.query("price < 200_000"),
    x="age_at_posting",
    y="price",
    title="Price vs Age at Posting",
    opacity=0.3,
    trendline="lowess",
    hover_data=["make_name", "model_name", "year", "listed_date"],
    labels={
        "price": "Price ($)",
        "age_at_posting": "Age at Posting (years)"
    },
    render_mode='webgl'
)
fig.show()

In [ ]:
# figure out what age of car on average costs 20k
avg_price_at_age = raw_carguru.groupby("age_at_posting").agg(
    mean_price = ("price", "mean"),
    num_vehicles = ("price", "count")
)

# get the row with avg price closest to 20k
print("Question: what age of vehicle might I expect to be able to get for $20k?\n")
print(f"The age of vehicles that have average resale prices closest to $20k is {avg_price_at_age.query('(mean_price > 16_000) & (mean_price < 24_000) & (age_at_posting < 15)').index.values} years old")
print(f"Avg prices and no. of vehicles are: ")
print(avg_price_at_age.query('(mean_price > 16_000) & (mean_price < 24_000) & (age_at_posting < 15)'))



In [ ]:
# release the raw_ads df
del raw_carguru
gc.collect()

## EDA Conclusions

1. A number of fields are missing >50% of there data, proposal to remove these columns
   - majority null values (>50%): bed, bed_height, bed_length, combine_fuel_economy, cabin, is_certified, is_cpo, is_oem_cpo, vehicle_damage_category
2. Other columns which are likely not informative to our analysis are:
   - vin - won't tell us unique info about a car, just a unique identifier
   - days on market - not useful as we don't know when the car was scraped
   - 
3. The price of cars shows patterns that majority of prices are around whole thousands of dollars, i.e. 12,000 not 12,500
4. The distribution of prices over age has a large spread
5. There are a large amount of outliers in the data, i.e. cars that are 20+ years old and have a price of $100,000
   - proposal is to ignore/remove these outliers as the analysis is not concerned about unique values but the standard ranges

# Analyzing Full Dataset

Initially the following columns are going to used for analysis:
- price
- listed_date
- description
- year
- mileage
- horsepower
- city
- fuel_type
- exterior_color

In [ ]:
def load_cargurus_dataset():

    cols_to_keep = [
        "make_name",
        "model_name",
        "year",
        "listed_date",
        "price",
        "mileage",
        # "description", 
        "major_options",
        "seller_rating",
        "horsepower",
        "fuel_type",
        "wheel_system",
        "price_cad",
        "price_currency",
        'exchange_rate_usd_to_cad'
        
    ]

    categorical_cols = [
        "make_name",
        "model_name",
        "fuel_type",
        "wheel_system",
    ]

    int_cols = [
        "year",
        "price",
        "horsepower",
    ]

    # raw_df = pd.read_csv(
    #     os.path.join(os.pardir, "data", "raw", "cargurus-vehicle-ads.csv"),
    #     usecols=cols_to_keep,
    #     parse_dates=["listed_date"],
    #     low_memory=True,
    # )

    raw_df = pd.read_parquet(
        os.path.join(os.pardir, "data", "processed", "processed-carguru-ads.parquet"),
        columns=cols_to_keep
    )

    for cat_col in categorical_cols:
        raw_df[cat_col] = raw_df[cat_col].astype("category")

    for int_col in int_cols:
        raw_df[int_col].fillna(-10, inplace=True)
        raw_df[int_col] = raw_df[int_col].astype("int32")

    return raw_df

In [ ]:
raw_ads = load_cargurus_dataset()

In [ ]:
# del raw_ads
# gc.collect()

In [ ]:
raw_ads.info(memory_usage="deep")

In [ ]:
def preprocess_carguru_ads(raw_ads):

    processed_ads = raw_ads.copy(deep=True)

    processed_ads = processed_ads.rename(
        columns={
            # old name : new name
            "price" : "price_usd", 
            "price_cad": "price",
            "make_name": "make",
            "model_name": "model"
        },
    )

    processed_ads["age_at_posting"] = (processed_ads.listed_date.dt.year - processed_ads.year).astype("int8")
    processed_ads["mileage_per_year"] = (processed_ads.mileage / processed_ads.age_at_posting).round(0)    

    # remove ads with prices > 500_000
    processed_ads = processed_ads.query("price < 500_000")

    # processed_ads["major_options"] = processed_ads.major_options.str.split(",").apply(lambda x: [opt.strip() for opt in x])

    return processed_ads

In [ ]:
ads = preprocess_carguru_ads(raw_ads)

In [ ]:
ads.head()

In [ ]:
def get_usd_to_cad_exchange(date):

    # check date is valid date object
    if not isinstance(date, dt.date):
        raise TypeError("date must be a datetime.date object")

    date_string = date.strftime("%Y-%m-%d")
    resp = requests.get(f"https://theforexapi.com/api/{date_string}/?symbols=CAD&base=USD", verify=False).json()

    return resp["rates"]["CAD"]

In [ ]:
# # for every unique date of listed_date get the exchange rate and convert to USD
# for d in ads.listed_date.dt.date.unique():
#     ads.loc[ads.listed_date.dt.date == d, "exchange_rate_usd_to_cad"] = get_usd_to_cad_exchange(d)

In [ ]:
# ads["price_cad"] = (ads.price * ads.exchange_rate_usd_to_cad).round(0).astype("int32")
# ads["price_currency"] = "CAD"

In [ ]:
# save ads to parquet file in data/processed
# ads.to_parquet(os.path.join(os.pardir, "data", "processed", "processed-carguru-ads.parq"))

In [ ]:
ads.info(memory_usage="deep")

In [ ]:
# plot how many ads there are by the top 30 make_name values
fig = px.histogram(
    # ads.loc[ads.make_name.isin(ads.make_name.value_counts().index[:15])],
    ads.loc[ads.make.isin(ads.make.value_counts().index[:15])].make.cat.remove_unused_categories(),
    x="make",
    title="Number of ads by Make (Top 15)",
    color="make",
    labels={"make": "Make"},
    color_discrete_sequence=px.colors.qualitative.Dark24,
    height=500,
    category_orders={"make": ads.make.value_counts().index[:15]}
)
fig.show()

In [ ]:
# plot distribution of cars with price < 200k
fig = px.histogram(
    ads.query("price < 200_000"),
    x="price",
    title="Distribution of Vehicle Prices < $200k",
    labels={
        "price": "Price ($)",
        "count": "Number of Ads"
    }
)
fig.add_vline(
    x=ads.query("price < 200_000").price.median(), 
    line_dash = 'dash', 
    line_color = 'firebrick',
    annotation_text = f" Median: ${ads.query('price < 200_000').price.median():.0f}",
)
fig.update_traces(
    xbins_size = 500
)
fig.show()

In [ ]:
# plot distribution of mileage per year
fig = px.histogram(
    ads.query("(mileage_per_year > 0) & (mileage_per_year < 100_000)"),
    x="mileage_per_year",
    title="Distribution of Mileage per Year",
    labels={
        "mileage_per_year": "Mileage per Year (mi)",
        "count": "Number of Ads"
    },
    # change bin width to 500
    nbins=200,
)
fig.add_vline(
    x=ads.query("(mileage_per_year > 0) & (mileage_per_year < 100_000)").mileage_per_year.median(), 
    line_dash = 'dash', 
    line_color = 'firebrick',
    annotation_text = f" Median: {ads.query('(mileage_per_year > 0) & (mileage_per_year < 100_000)').mileage_per_year.median():.0f} mi/year",
)
fig.show()

In [ ]:
# groupby age at posting and calculate median price and 95/5 quantiles
agg_ads = ads.groupby("age_at_posting", as_index=False).agg(
    median_price=("price", "median"),
    price_95 = ("price", lambda x: x.quantile(0.95)),
    price_05 = ("price", lambda x: x.quantile(0.05)),
).query("(price_95 < 200_000) & (age_at_posting < 25)")

fig = go.Figure([
    go.Scatter(
        name='Median Price',
        x=agg_ads.age_at_posting,
        y=agg_ads.median_price,
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper 95%',
        x=agg_ads.age_at_posting,
        y=agg_ads.price_95,
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower 5%',
        x=agg_ads.age_at_posting,
        y=agg_ads.price_05,
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    )
])
fig.update_layout(
    yaxis_title='Price (CAD)',
    title='Car Price by Age at Posting',
    hovermode="x",
    xaxis_range=[0,20]
)
fig.show()

In [ ]:
def plot_price_distribution_by_age(ads_df, makes, models, display=True):

    # for each value in make/model combo, plot the price distribution
    color_map = [
        "rgba(31, 119, 180)",
        "rgba(255, 127, 14)",
        "rgba(44, 160, 44)",
        "rgba(214, 39, 40)",
    ]
    num_ads = 0

    fig = go.Figure()
    for i in range(len(makes)):

        rgb_color = color_map[i]

        make = makes[i]
        model = models[i]
        # groupby age at posting and calculate median price and 95/5 quantiles
        agg_ads = ads_df.query("make == @make & model == @model").groupby("age_at_posting", as_index=False).agg(
            median_price=("price", "median"),
            price_95 = ("price", lambda x: x.quantile(0.95)),
            price_05 = ("price", lambda x: x.quantile(0.05)),
        ).query("(price_95 < 200_000) & (age_at_posting < 25)")

        num_ads += len(ads_df.query("make == @make & model == @model"))

        # add line trace for median price
        fig.add_trace(
            go.Scatter(
                name=f"{make} {model} Median Price",
                x=agg_ads.age_at_posting,
                y=agg_ads.median_price,
                mode='lines',
                line=dict(color=rgb_color.replace("a", "")),
            )
        )

        # add line trace for 95th percentile
        fig.add_trace(
            go.Scatter(
                name=f"{make} {model} Upper 95%",
                x=agg_ads.age_at_posting,
                y=agg_ads.price_95,
                mode='lines',
                marker=dict(color="#444"),
                line=dict(width=0),
                showlegend=False
            )
        )

        # add line trace for 5th percentile
        fig.add_trace(
            go.Scatter(
                name=f"{make} {model} Lower 5%",
                x=agg_ads.age_at_posting,
                y=agg_ads.price_05,
                mode='lines',
                line=dict(width=0),
                fillcolor=rgb_color.replace(")", ", 0.3)"),
                fill='tonexty',
                marker=dict(color="#444"),
                showlegend=False
            )
        )

    
    fig.update_layout(
        yaxis_title='Price (CAD)',
        xaxis_title='Age at Posting (years)',
        title=f'Car Price by Age of Vehicle for: {", ".join([f"{ma} {m}" for ma, m in zip(makes, models)])}<br><sup>From {num_ads} Ads</sup>',
        hovermode="x",
        xaxis_range=[0,20]
    )
    
    if display is True:
        fig.show()
    else:
        return fig

In [ ]:
def plot_price_depreciation_by_age(ads_df, makes, models, display=True):

    # for each value in make/model combo, plot the price distribution
    color_map = [
        "rgba(31, 119, 180)",
        "rgba(255, 127, 14)",
        "rgba(44, 160, 44)",
        "rgba(214, 39, 40)",
    ]
    num_ads = 0

    fig = go.Figure()
    for i in range(len(makes)):

        rgb_color = color_map[i]

        make = makes[i]
        model = models[i]
        # groupby age at posting and calculate median price and 95/5 quantiles
        agg_ads = ads_df.query("make == @make & model == @model").groupby("age_at_posting", as_index=False).agg(
            median_price=("price", "median"),
            # price_95 = ("price_cad", lambda x: x.quantile(0.95)),
            # price_05 = ("price_cad", lambda x: x.quantile(0.05)),
        ).query("(age_at_posting < 25)") #(price_95 < 200_000) & 

        agg_ads["price_drop"] = agg_ads.median_price.diff(1).fillna(0) * -1

        num_ads += len(ads_df.query("make == @make & model == @model"))

        # add line trace for median price
        fig.add_trace(
            go.Scatter(
                name=f"{make} {model} Median Price",
                x=agg_ads.age_at_posting,
                y=agg_ads.price_drop,
                mode='lines',
                line=dict(color=rgb_color.replace("a", "")),
            )
        )
    
    fig.update_layout(
        yaxis_title='Price Decrease ($CAD)',
        xaxis_title='Age at Posting (years)',
        title=f'Car Depreciation Rates for: {", ".join([f"{ma} {m}" for ma, m in zip(makes, models)])}<br><sup>From {num_ads} Ads</sup>',
        hovermode="x",
        xaxis_range=[1,20],
        yaxis_range=[0,None]
    )
    
    if display is True:
        fig.show()
    else:
        return fig

In [ ]:
ads.query("make == 'Toyota'").model.unique()

In [ ]:
makes = ["Subaru", "Toyota", "Honda"] 
models = [ "Outback", "RAV4", "CR-V"]

In [ ]:
plot_price_distribution_by_age(ads, makes, models)
plot_price_depreciation_by_age(ads, makes, models)

In [ ]:
ads.query("make == 'Subaru' & model == 'Outback'").model.count()

# Combine Carguru & Kijiji Data

In [ ]:
import src.data.cargurus as cargurus

In [ ]:
cargurus.preprocess_raw_cargurus_data(nrows = 1000, export = False)